### NIH Analaysis 

#### sql queries with aggregation

##### counts of completed studies based off of collaborator agency class 

In [38]:
%%bigquery 

select collaborator_agency_class, count(overall_status) as completed
from nih_modeled.clinical_studies_main_Beam_DF csm join nih_modeled.collaborators_Beam_DF col
on csm.nct_number = col.nct_number 
where overall_status = 'Completed'
group by collaborator_agency_class
order by completed desc

,collaborator_agency_class,completed
0,Other,18486
1,Industry,10370
2,NIH,8913
3,U.S. Fed,1035


##### counts of terminated studies based off of collaborator agency class

In [37]:
%%bigquery 

select collaborator_agency_class, count(overall_status) as terminated
from nih_modeled.clinical_studies_main_Beam_DF csm join nih_modeled.collaborators_Beam_DF col
on csm.nct_number = col.nct_number 
where overall_status = 'Terminated'
group by collaborator_agency_class
order by terminated desc

,collaborator_agency_class,terminated
0,Industry,1710
1,Other,1641
2,NIH,885
3,U.S. Fed,86


##### counts the amount of intervention per type based off having an enrollment greater than 1000 as well as more than one arm group

In [35]:
%%bigquery 

select intervention_type, count(intervention_type) as intervention_count
from nih_modeled.clinical_studies_main_Beam_DF csm join nih_modeled.interventions_Beam_DF inter
on csm.nct_number = inter.nct_number 
where enrollment > 1000
group by intervention_type
having count(arm_group_label) > 1
order by intervention_count desc

,intervention_type,intervention_count
0,Drug,3536
1,Other,1887
2,Behavioral,1547
3,Procedure,855
4,Biological,788
5,Device,675
6,Dietary Supplement,153
7,Radiation,108
8,Genetic,55


##### amount of safety issues based off of type of study 

In [39]:
%%bigquery 

select type, count(safety_issue) as safety_issue 
from nih_modeled.clinical_results_Beam_DF 
where safety_issue = True 
group by type 
order by safety_issue desc

,type,safety_issue
0,Primary,2156
1,Secondary,1809
2,Other Pre-specified,123
3,Post-Hoc,10


##### amount of studies without safety issues based off of type of study 

In [42]:
%%bigquery 

select type, count(safety_issue) as non_safety_issue 
from nih_modeled.clinical_results_Beam_DF 
where safety_issue = False 
group by type 
order by non_safety_issue desc

,type,non_safety_issue
0,Secondary,10292
1,Primary,9080
2,Other Pre-specified,360
3,Post-Hoc,54


##### studies unique to females by phase 

In [56]:
%%bigquery 

select phase, count(eligibility_gender) as female_count
from nih_modeled.clinical_studies_main_Beam_DF csm join nih_modeled.eligibility_Beam_DF el
on csm.nct_number = el.nct_number 
where eligibility_gender = 'Female'
group by phase
having phase <> 'N/A'
order by female_count desc

,phase,female_count
0,Phase 2,3586
1,Phase 3,2698
2,Phase 4,2182
3,Phase 1,1384
4,Phase 1/Phase 2,635
5,Phase 2/Phase 3,454
6,Phase 0,241


#### create aggregate query views

In [58]:
%%bigquery 

create or replace view nih_modeled.v_completed_cases as
select collaborator_agency_class, count(overall_status) as completed
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm join `probable-pager-266720.nih_modeled.collaborators_Beam_DF` col
on csm.nct_number = col.nct_number 
where overall_status = 'Completed'
group by collaborator_agency_class
order by completed desc

""


In [59]:
%%bigquery 

create or replace view nih_modeled.v_terminated_cases as
select collaborator_agency_class, count(overall_status) as terminated
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm join `probable-pager-266720.nih_modeled.collaborators_Beam_DF` col
on csm.nct_number = col.nct_number 
where overall_status = 'Terminated'
group by collaborator_agency_class
order by terminated desc

""


#### sql queries with subqueries 

##### query joins the interventions and clinical studies main tables on nct_number in order to find all the studies where a drug is the intervention type

In [15]:
%%bigquery 

select csm.* 
from nih_modeled.clinical_studies_main_Beam_DF csm
where csm.nct_number in 
    (select nct_number
     from nih_modeled.interventions_Beam_DF int
     where int.intervention_type = 'Drug') 
order by lead_sponsor_agency_class

,nct_number,org_study_id,secondary_id,official_title,brief_summary,overall_status,enrollment,enrollment_type,start_date,competion_date,...,phase,study_type,study_design,first_recieved_date,verification_date,lead_sponsor_agency,lead_sponsor_agency_class,official_full_name,overall_official_affiliation,serialid
0,NCT02592941,MP-104-CL-037,None,"An Open Label, Expanded Access Protocol Intend...",\n The expanded access program will provi...,Available,NaN,None,None,None,...,N/A,Expanded Access,N/A,2015-10-29,2016-09-01,"Marathon Pharmaceuticals, LLC",Industry,None,None,166051
1,NCT01947608,CLDK378A2402,None,"An Open-label, Multi-center, Expanded Treatmen...","\n Novartis-sponsored, open-label, multi-...",Available,NaN,None,None,None,...,N/A,Expanded Access,N/A,2013-09-17,2016-03-01,Novartis Pharmaceuticals,Industry,None,None,191064
2,NCT02197650,GRASPALL 2012-10-EAP,None,"Expanded Access Program for a Multicentre, Ope...",\n Early Access Program to provide L-aspa...,Available,NaN,None,2013-02-01,2016-12-01,...,Phase 1,Expanded Access,N/A,2014-05-07,2016-03-01,ERYtech Pharma,Industry,"Yves BERTRAND, Pr","IHOP, Lyon",211159
3,NCT02433314,547-SSE-302,None,"An Open Label, Expanded Access Protocol to Eva...","\n This is an open-label, expanded access...",Available,NaN,None,None,None,...,N/A,Expanded Access,N/A,2015-04-29,2015-08-01,Sage Therapeutics,Industry,"Henrikas Vaitkevicius, MD",Brigham and Women's Hospital,110962
4,NCT02186249,CA209-218,None,Expanded Access Program With Nivolumab (BMS-93...,\n The purpose of this study is to provid...,Available,NaN,None,None,None,...,N/A,Expanded Access,N/A,2014-07-08,2016-05-01,Bristol-Myers Squibb,Industry,Bristol-Myers Squibb,Bristol-Myers Squibb,185234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97640,NCT02084615,2013-0976,None,"A Prospective, Nonrandomized, Study Comparing ...",\n The purpose of this study is to determ...,Enrolling by invitation,300.0,Anticipated,2014-08-01,2017-06-01,...,N/A,Interventional,"Allocation: Non-Randomized, Endpoint Classific...",2014-03-09,2015-06-01,Jesse Brown VA Medical Center,U.S. Fed,"Joseph M Vitello, MD",Jesse Brown VA Medical Center,38600
97641,NCT02527083,PRO00000878,None,Influence of Anesthetic Technique on Acute and...,\n Research suggests that the type of ane...,Enrolling by invitation,48.0,Anticipated,2015-01-01,2018-06-01,...,Phase 4,Interventional,"Allocation: Randomized, Endpoint Classificatio...",2015-01-27,2016-11-01,VA Pittsburgh Healthcare System,U.S. Fed,None,None,23173
97642,NCT02214615,SW0023,None,Pilot Study on the Response of Inherited Eryth...,\n This research study is designed to inv...,Enrolling by invitation,2.0,Anticipated,2014-04-01,2016-06-01,...,Phase 4,Interventional,"Allocation: Randomized, Endpoint Classificatio...",2014-08-10,2015-07-01,VA Connecticut Healthcare System,U.S. Fed,"Stephen Waxman, MD, PhD",VAMC West Haven and Yale University,86799
97643,NCT02458781,398092-1,None,Antibiotic Treatment Trial for Small Intestina...,"\n This is a randomized, double-blind, pl...",Enrolling by invitation,180.0,Anticipated,2015-01-01,2018-06-01,...,Phase 4,Interventional,"Allocation: Randomized, Endpoint Classificatio...",2015-04-10,2015-05-01,San Antonio Military Medical Center,U.S. Fed,"Karin S Gilkison, MD, MPH",Gastroenterology Fellow,137641


##### query joins interventions and clinical studies main tables on nct_number in order to find studies that have less than the average enrollment amount 

In [17]:
%%bigquery

select csm.* 
from nih_modeled.interventions_Beam_DF int join nih_modeled.clinical_studies_main_Beam_DF csm
on int.nct_number = csm.nct_number
where csm.enrollment < 
    (select avg(enrollment) 
     from nih_modeled.clinical_studies_main_Beam_DF)
order by lead_sponsor_agency_class

,nct_number,org_study_id,secondary_id,official_title,brief_summary,overall_status,enrollment,enrollment_type,start_date,competion_date,...,phase,study_type,study_design,first_recieved_date,verification_date,lead_sponsor_agency,lead_sponsor_agency_class,official_full_name,overall_official_affiliation,serialid
0,NCT00789841,M-20080121,None,Gastrointestinal Motility in Patients With Neu...,\n We will study the total gastrointestin...,Completed,13,Actual,2008-09-01,2010-05-01,...,N/A,Observational,N/A,2008-11-11,2010-10-01,University of Aarhus,Other,None,None,182891
1,NCT02332265,144-PRJ51ZT,R01HD055868,"Pathways Linking Poverty, Food Insecurity, and...",\n The purpose of this study is to evalua...,Completed,1901,Actual,2009-02-01,2012-04-01,...,N/A,Observational,N/A,2014-12-21,2015-01-01,"University of Wisconsin, Milwaukee",Other,"Lance Weinhardt, PhD","University of Wisconsin, Milwaukee, Zilber Sch...",131902
2,NCT00373035,CCCWFU-85A03,CDR0000490117CCCWFU-BG03-628,Biomarkers of Prostate Cancer,\n RATIONALE: Collecting and storing samp...,Completed,60,Anticipated,2006-02-01,None,...,N/A,Observational,N/A,2006-09-06,2013-06-01,Wake Forest University Health Sciences,Other,"Frank M. Torti, MD, MPH",Wake Forest University Health Sciences,135811
3,NCT00439478,171074,None,The Dental Safety Profile of High-Dose Radioio...,\n We aim to assess the incidence of oral...,Completed,202,None,2004-09-01,2006-09-01,...,Phase 4,Observational,N/A,2007-02-22,2007-10-01,"University Hospital, Basel, Switzerland",Other,"Beat Müller, MD","Division of Endocrinology, University Hospital...",196648
4,NCT00286702,HEC 01/166,None,None,"\n After a stroke, many people develop co...",Completed,63,None,2002-10-01,2004-09-01,...,N/A,Interventional,N/A,2006-02-01,2006-01-01,University of Western Sydney,Other,"Natasha Lannin, BSc(OT)Anne Cusick, PhD",University of Western SydneyUniversity of West...,50409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201829,NCT01602861,Eudra CT: 2011-002243-98,None,The Effects of Spironolactone on Calcineurin I...,\n The purpose of this study is to assess...,Enrolling by invitation,170,Anticipated,2013-02-01,2017-09-01,...,Phase 4,Interventional,"Allocation: Randomized, Endpoint Classificatio...",2012-05-17,2014-01-01,Odense University Hospital,Other,"Claus Bistrup, MD, ph.d.","Dep. of Nephrology, Odense University Hospital",30971
201830,NCT00885651,13-5-23-9-3-8-15,EudraCTnr. 2008-001908-23,"A Randomised, Double Blinded, Crossover Study ...",\n Heart failure is a frequent disease in...,Enrolling by invitation,30,Anticipated,2008-09-01,2009-12-01,...,Phase 4,Interventional,"Allocation: Randomized, Endpoint Classificatio...",2009-04-20,2009-04-01,"Rigshospitalet, Denmark",Other,"Henrik HP Enghusen Poulsen, MD","Rigshospitalet, Universityhospital of Copenhagen",214417
201831,NCT03008356,16-005732,None,L-Carnitine Supplementation With and Without H...,\n We hypothesize that supplementation wi...,Enrolling by invitation,90,Anticipated,2017-01-01,2017-11-01,...,N/A,Interventional,"Allocation: Randomized, Endpoint Classificatio...",2016-12-28,2016-12-01,Mayo Clinic,Other,Roberto P Benzo,Mayo Clinic,141174
201832,NCT02517502,STUDY00002415,None,Docosahexaenoic Acid (DHA) To Prevent Developm...,\n The purpose of this study is to determ...,Enrolling by invitation,50,Anticipated,2015-07-01,2017-12-01,...,Phase 0,Interventional,"Allocation: Randomized, Endpoint Classificatio...",2015-07-08,2016-05-01,"Carol Fabian, MD",Other,"Carol Fabian, MD",University of Kansas Medical Center,133577


##### query selects studies that had safety issues in both primary outcomes and secondary outcomes 

In [12]:
%%bigquery 

select csm.*
from nih_modeled.clinical_studies_main_Beam_DF csm
where csm.nct_number in 
    (select po.nct_number
    from nih_modeled.primary_outcomes_Beam_DF po inner join nih_modeled.secondary_outcomes_Beam_DF so 
    on po.nct_number = so.nct_number 
    where (po.safety_issue = True) and (so.safety_issue = True))
order by nct_number

,nct_number,org_study_id,secondary_id,official_title,brief_summary,overall_status,enrollment,enrollment_type,start_date,competion_date,...,phase,study_type,study_design,first_recieved_date,verification_date,lead_sponsor_agency,lead_sponsor_agency_class,official_full_name,overall_official_affiliation,serialid
0,NCT00000134,NEI-33,U10EY008057U01AI027668,Cytomegalovirus Retinitis Retreatment Trial,\n To compare the relative merits of thre...,Completed,279.0,Actual,1992-12-01,1995-03-01,...,Phase 3,Interventional,"Allocation: Randomized, Endpoint Classificatio...",1999-09-23,2015-08-01,Johns Hopkins Bloomberg School of Public Health,Other,None,None,151016
1,NCT00000135,NEI-34,None,Monoclonal Antibody CMV Retinitis Trial (MACRT),\n To evaluate the efficacy and safety of...,Completed,209.0,Actual,1995-09-01,1996-08-01,...,Phase 2/Phase 3,Interventional,"Allocation: Randomized, Endpoint Classificatio...",1999-09-23,2015-07-01,Johns Hopkins Bloomberg School of Public Health,Other,None,None,14148
2,NCT00000136,NEI-35,U10EY008057,Foscarnet-Ganciclovir CMV Retinitis Trial,\n To evaluate the relative safety and ef...,Completed,234.0,Actual,1990-03-01,1991-10-01,...,Phase 3,Interventional,"Allocation: Randomized, Endpoint Classificatio...",1999-09-23,2015-07-01,Johns Hopkins Bloomberg School of Public Health,Other,"Curtis Meinert, PhD",Johns Hopkins University,220092
3,NCT00000142,NEI-41,None,Studies of the Ocular Complications of AIDS (S...,\n To test and evaluate the efficacy and ...,Completed,64.0,Actual,1994-04-01,1996-02-01,...,Phase 2/Phase 3,Interventional,"Allocation: Randomized, Endpoint Classificatio...",1999-09-23,2015-07-01,Johns Hopkins Bloomberg School of Public Health,Other,"Douglas Jabs, MD",Icahn School of Medicine at Mount Sinai,55387
4,NCT00000143,NEI-42,None,Studies of Ocular Complications of AIDS (SOCA)...,\n To compare the newest CMV retinitis dr...,Completed,61.0,Actual,1997-05-01,2000-06-01,...,Phase 3,Interventional,"Allocation: Randomized, Endpoint Classificatio...",1999-09-23,2015-07-01,Johns Hopkins Bloomberg School of Public Health,Other,"Douglas Jabs, MD",SOCA Chairman's Office,171088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40094,NCT03024359,CAAE: 60698716.1.0000.5404,None,Effect of Monounsaturated Fatty Acids Consumpt...,\n Considering the high prevalence of obe...,Recruiting,80.0,Anticipated,2017-01-01,2018-08-01,...,Phase 0,Interventional,"Allocation: Non-Randomized, Intervention Model...",2017-01-11,2017-01-01,"University of Campinas, Brazil",Other,"Milena Monfort-Pires, PhDLicio A Velloso, Prof...",University of CampinasUniversity of Campinas,96375
40095,NCT03024385,2016-0116,None,Interconception Care at the University of Miss...,\n This model will utilize tools from the...,Not yet recruiting,1625.0,Anticipated,2017-03-01,2018-11-01,...,N/A,Observational,"Observational Model: Cohort, Time Perspective:...",2017-01-13,2016-10-01,University of Mississippi Medical Center,Other,"Nneka A Holder, MD, MPH",University of Mississippi Medical Center,219100
40096,NCT03024450,ZS-ON-08,None,Efficacy and Prognostic Factors of Trastuzumab...,\n Trastuzumab plus chemotherapy is an ef...,Completed,98.0,Actual,2012-01-01,2016-12-01,...,N/A,Observational [Patient Registry],"Observational Model: Cohort, Time Perspective:...",2017-01-02,2017-01-01,Shanghai Zhongshan Hospital,Other,"Tianshu / Liu, Doctor",Shanghai Zhongshan Hospital,234717
40097,NCT03024489,RA2016001,None,A Phase I/II Dose Escalation Study of the CDK4...,\n Cyclin D kinase 4 (CDK4) is a key regu...,Not yet recruiting,33.0,Anticipated,2017-02-01,2026-12-01,...,Phase 1/Phase 2,Interventional,Endpoint Classification: Safety/Efficacy Study...,2016-11-28,2017-01-01,Mahidol University,Other,"Nuttapong Ngamphaiboon, MD","Faculty of Medicine, Ramathibodi Hospital, Mah...",74896


##### query counts the amount of studies by their set eligibility gender and then groups them based off of the typ eof lead sponsor agency class 

In [44]:
%%bigquery 

(select csm.lead_sponsor_agency_class, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Male' and lead_sponsor_agency_class = 'Industry') as males_count, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Female' and lead_sponsor_agency_class = 'Industry') as females_count, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'All' and lead_sponsor_agency_class = 'Industry') as all_count, 
from nih_modeled.clinical_studies_main_Beam_DF csm join nih_modeled.eligibility_Beam_DF e
on csm.nct_number = e.nct_number 
where lead_sponsor_agency_class = 'Industry'
group by lead_sponsor_agency_class

union all 

select csm.lead_sponsor_agency_class, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Male' and lead_sponsor_agency_class = 'NIH') as males_count, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Female' and lead_sponsor_agency_class = 'NIH') as females_count, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'All' and lead_sponsor_agency_class = 'NIH') as all_count, 
from nih_modeled.clinical_studies_main_Beam_DF csm join nih_modeled.eligibility_Beam_DF e
on csm.nct_number = e.nct_number 
where lead_sponsor_agency_class = 'NIH'
group by lead_sponsor_agency_class

union all 

select csm.lead_sponsor_agency_class, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Male' and lead_sponsor_agency_class = 'U.S. Fed') as males_count, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Female' and lead_sponsor_agency_class = 'U.S. Fed') as females_count, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'All' and lead_sponsor_agency_class = 'U.S. Fed') as all_count, 
from nih_modeled.clinical_studies_main_Beam_DF csm join nih_modeled.eligibility_Beam_DF e
on csm.nct_number = e.nct_number 
where lead_sponsor_agency_class = 'U.S. Fed'
group by lead_sponsor_agency_class

union all 

select csm.lead_sponsor_agency_class, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Male' and lead_sponsor_agency_class = 'Other') as males_count, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Female' and lead_sponsor_agency_class = 'Other') as females_count, 
    (select count(eligibility_gender) 
     from nih_modeled.eligibility_Beam_DF e join nih_modeled.clinical_studies_main_Beam_DF csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'All' and lead_sponsor_agency_class = 'Other') as all_count, 
from nih_modeled.clinical_studies_main_Beam_DF csm join nih_modeled.eligibility_Beam_DF e
on csm.nct_number = e.nct_number 
where lead_sponsor_agency_class = 'Other'
group by lead_sponsor_agency_class)

,lead_sponsor_agency_class,males_count,females_count,all_count
0,Other,6618,17916,130083
1,NIH,609,777,8481
2,Industry,4536,3971,57706
3,U.S. Fed,163,144,2744


#### create subquery view 

In [51]:
%%bigquery

create or replace view nih_modeled.v_eligibility_gender_count as

select csm.lead_sponsor_agency_class, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Male' and lead_sponsor_agency_class = 'Industry') as males_count, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Female' and lead_sponsor_agency_class = 'Industry') as females_count, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'All' and lead_sponsor_agency_class = 'Industry') as all_count
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm join `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e
on csm.nct_number = e.nct_number 
where lead_sponsor_agency_class = 'Industry'
group by lead_sponsor_agency_class

union all 

select csm.lead_sponsor_agency_class, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Male' and lead_sponsor_agency_class = 'NIH') as males_count, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Female' and lead_sponsor_agency_class = 'NIH') as females_count, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'All' and lead_sponsor_agency_class = 'NIH') as all_count
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm join `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e
on csm.nct_number = e.nct_number 
where lead_sponsor_agency_class = 'NIH'
group by lead_sponsor_agency_class

union all 

select csm.lead_sponsor_agency_class, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Male' and lead_sponsor_agency_class = 'U.S. Fed') as males_count, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Female' and lead_sponsor_agency_class = 'U.S. Fed') as females_count, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'All' and lead_sponsor_agency_class = 'U.S. Fed') as all_count
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm join `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e
on csm.nct_number = e.nct_number 
where lead_sponsor_agency_class = 'U.S. Fed'
group by lead_sponsor_agency_class

union all 

select csm.lead_sponsor_agency_class, 
    (select count(eligibility_gender)  
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Male' and lead_sponsor_agency_class = 'Other') as males_count, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'Female' and lead_sponsor_agency_class = 'Other') as females_count, 
    (select count(eligibility_gender) 
     from `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e join `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
     on e.nct_number = csm.nct_number
     where eligibility_gender = 'All' and lead_sponsor_agency_class = 'Other') as all_count
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm join `probable-pager-266720.nih_modeled.eligibility_Beam_DF` e
on csm.nct_number = e.nct_number 
where lead_sponsor_agency_class = 'Other'
group by lead_sponsor_agency_class

""
